# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-boundc

    Updating registry at `~/.julia/registries/General`
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/utils.jl:36
┌ Error: curl_easy_setopt: 48
└ @ Downloads.Curl /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Downloads/src/Curl/uti

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [3]:
# ADAPTED
function TestsSondabilite_relaxlin(model2, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(!model2.feasable)#Test de faisabilite
        TA=true
        println("TA")
    elseif(model2.bound <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif(checkResolution(model2)) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objective(model2) >= BestProfit)
            Bestsol = value.(model2.variables)
            #BestProfit=value(benef)
            BestProfit = objective(model2)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_relaxlin (generic function with 1 method)

### Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:
# ADAPTED
function SeparerNoeud_relaxlin(model2, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche  

    local nextVar
    if model2.boundComputation == "fks"
        for (i, var) in enumerate(model2.variables)
            if var.free && (abs(round(value(var), digits=0) - value(var) ) >= 10^-5)
                setVar(model2, i, 1.0)
                nextVar = var
                break
            end
        end
    elseif model2.boundComputation == "simple"
        for (i, var) in enumerate(model2.variables)
            if var.free
                setVar(model2, i, 1.0)
                nextVar = var
                break
            end
        end
    end

    push!(listvars, nextVar) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals, 1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(model2, listvars, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listvars)>= 1)
        #go back to parent node
        var=pop!(listvars)
        theval=pop!(listvals)
        tmp=pop!(listnodes)
        freeVar(model2, var, 0.0, 1.0)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listvars)>= 1))
            var=pop!(listvars)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
            freeVar(model2, var, 0.0, 1.0)
        end
        if theval==1.0
            setVar(model2, var, 0.0)
            push!(listvars,var)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, listnodes, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [5]:
# CUSTOM IMPLEMENTATION

mutable struct varDecision
    value::Float64
    index::Int64
    free::Bool
    minValue::Float64
    maxValue::Float64
end

function value(var)
    return var.value
end

function name(var)
    return string("x[", var.index, "]")
end

mutable struct myModel
    unsortedVariables::Array{varDecision, 1}
    variables::Array{varDecision, 1}
    dof::Int64
    prices::Array{Int64, 1}
    weights::Array{Int64, 1}
    capacity::Int64
    boundComputation::String
    feasable::Bool
    bound::Float64
    #weight::Float64
    #profit::Float64
end


function myPrint(model)
    print("Max")
    for (index, var) in enumerate(model.unsortedVariables)
        print(" + ", model.prices[index], " x[", index, "]")
    end
    println("")
    println("Subject to")
    for (index, var) in enumerate(model.unsortedVariables)
        print(" + ", model.weights[index], " x[", index, "]")
    end
    println(" <= ", model.capacity)
    for (index, var) in enumerate(model.unsortedVariables)
        println(" x[", index, "] >= ", var.minValue)
    end
    for (index, var) in enumerate(model.unsortedVariables)
        println(" x[", index, "] <= ", var.maxValue)
    end
    println("")
end

function objective(model)
    return sum([var.value * model.prices[var.index] for var in model.variables])
end

function knownProfit(model)
    return mapreduce(var -> (var.free ? 0. : var.value) * model.prices[var.index], +, model.variables, init = 0)
end

function knownWeight(model)
    return mapreduce(var -> (var.free ? 0. : var.value) * model.weights[var.index], +, model.variables, init = 0)
end

function solveLinearRelax(model)
    residualCapacity = model.capacity - knownWeight(model)
    feasable = residualCapacity >= 10^-5
    bound = -Inf
    if feasable
        if model.boundComputation == "fks"
            for (index, var) in enumerate(model.variables)
                if var.free
                    if residualCapacity <= 10^-5
                        var.value = 0.0
                    else
                        weight = model.weights[var.index]
                        if weight <= residualCapacity
                            var.value = 1.0
                            residualCapacity -= weight
                        else
                            var.value = residualCapacity / weight
                            residualCapacity = 0.0
                        end
                    end
                end
            end
            
        bound = objective(model)
        
        elseif model.boundComputation == "simple"
            bound = knownProfit(model)
            for var in model.variables
                if var.free
                    bound += residualCapacity * (model.prices[var.index] / model.weights[var.index])
                    break
                end
            end
        end

    end
    
    model.feasable = feasable
    model.bound = bound
    
end


function checkResolution(model)
    easyProblem = false
    integerConstraint = false
    if model.boundComputation == "simple"
        easyProblem = (model.dof == 1)
        if easyProblem
            for (index, var) in enumerate(model.variables)
                if var.free
                    residualCapacity = model.capacity - knownWeight(model)
                    weight = model.weights[var.index]
                    var.value = weight <=  residualCapacity  ? 1.0 : 0.0
                    break
                end
            end
        end
    elseif model.boundComputation == "fks"
        varsbin = model.variables
        integerConstraint = (prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) 
        )
    end
    
    return easyProblem || integerConstraint
end

function setVar(model, index::Int64, value)
    var = model.variables[index]
    setVar(model, var, value)
end

function setVar(model, var::varDecision, value)
    var.free = false
    var.value = value
    var.minValue = value
    var.maxValue = value
    model.dof -= 1
end


function freeVar(model, var, minValue, maxValue)
    var.free = true
    var.minValue = minValue
    var.maxValue = maxValue
    model.dof += 1
end

freeVar (generic function with 1 method)

In [6]:
# ADAPTED
function CreationModeleLP(price, weight, capacity, boundComputation)
# ROOT NODE
    
    perm = sortperm(price ./ weight, rev = true)
    N = length(price)
    vars = Array{varDecision, 1}(undef, N)
    for i = 1:N
        vars[i] = varDecision(0.0, i, true, 0.0, 1.0)
    end
    model2 = myModel(vars, view(vars, perm), N, price, weight, capacity, boundComputation, false, -Inf)
    
    return model2
end


CreationModeleLP (generic function with 1 method)

### Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [7]:
# ADAPTED
function SolveKnapInstance(filename)

    if true
        price, weight, capacity = readKnaptxtInstance(filename)
        model2 = CreationModeleLP(price, weight, capacity, "fks")
    
        #create the structure to memorize the search tree for visualization at the end
        trParentnodes=Int64[] #will store orig node of arc in search tree
        trChildnodes=Int64[] #will store destination node of arc in search tree
        trNamenodes=[] #will store names of nodes in search tree
         
        #intermediate structure to navigate in the search tree
        listvars=[]
        listvals=[]
        listnodes=[]

        BestProfit=-1
        Bestsol=[]

        current_node_number=0
        stop = false

        while(!stop)

            println("\nNode number ", current_node_number, ": \n-----")
           # myPrint(model2)

            #Update the search tree
            push!(trNamenodes,current_node_number+1) 
            if(length(trNamenodes)>=2)
                push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
                push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
            end
            push!(listnodes, current_node_number)


            #print("Solve model2 to compute the bounds of the current node: start ... ")
            solveLinearRelax(model2)
            
            #println("... end")

            #print("\nSolution relax lin"); 
            
            if(!model2.feasable)#(has_values(model2))
                print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
            else
                if model2.boundComputation == "fks"
                   # [print("\t", name(v),"=",value(v)) for v in model2.unsortedVariables]
                elseif model2.boundComputation == "simple"
                   # print(" : Objective function bound : ", model2.bound)
                end
                    
            end
            
            #println(" "); println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            
            TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_relaxlin(model2, BestProfit, Bestsol)

            is_node_sondable = TA || TO || TR

            if(!is_node_sondable)
                listvars, listvals = SeparerNoeud_relaxlin(model2, listvars, listvals)
            else
                listvars, listvals, listnodes, stop = ExplorerAutreNoeud_relaxlin(model2, listvars, listvals, listnodes)
            end

            current_node_number = current_node_number + 1
        end

        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

        return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes
    end

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [ ]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("knapPI_16_100_1000_5_-8569.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
-----
non sondable

Node number 1: 
-----
non sondable

Node number 2: 
-----
non sondable

Node number 3: 
-----
non sondable

Node number 4: 
-----
non sondable

Node number 5: 
-----
non sondable

Node number 6: 
-----
non sondable

Node number 7: 
-----
non sondable

Node number 8: 
-----
non sondable

Node number 9: 
-----
non sondable

Node number 10: 
-----
non sondable

Node number 11: 
-----
non sondable

Node number 12: 
-----
non sondable

Node number 13: 
-----
non sondable

Node number 14: 
-----
non sondable

Node number 15: 
-----
non sondable

Node number 16: 
-----
non sondable

Node number 17: 
-----
non sondable

Node number 18: 
-----
non sondable

Node number 19: 
-----
non sondable

Node number 20: 
-----
non sondable

Node number 21: 
-----
non sondable

Node number 22: 
-----
non sondable

Node number 23: 
-----
non sondable

Node number 24: 
-----
non sondable

Node number 25: 
-----
non sondable

Node number 26: 
-----
non sondable

Node numbe

Node number 308: 
-----
TO

Node number 309: 
-----
TO

Node number 310: 
-----
TO

Node number 311: 
-----
TO

Node number 312: 
-----
TO

Node number 313: 
-----
TO

Node number 314: 
-----
TO

Node number 315: 
-----
TO

Node number 316: 
-----
TO

Node number 317: 
-----
TO

Node number 318: 
-----
TO

Node number 319: 
-----
TO

Node number 320: 
-----
TO

Node number 321: 
-----
TO

Node number 322: 
-----
TO

Node number 323: 
-----
TO

Node number 324: 
-----
TO

Node number 325: 
-----
TO

Node number 326: 
-----
TO

Node number 327: 
-----
TO

Node number 328: 
-----
TO

Node number 329: 
-----
TO

Node number 330: 
-----
TO

Node number 331: 
-----
TO

Node number 332: 
-----
TO

Node number 333: 
-----
non sondable

Node number 334: 
-----
TO

Node number 335: 
-----
TO

Node number 336: 
-----
non sondable

Node number 337: 
-----
TO

Node number 338: 
-----
non sondable

Node number 339: 
-----
TO

Node number 340: 
-----
TO

Node number 341: 
-----
non sondable

Node num

Node number 613: 
-----
TO

Node number 614: 
-----
non sondable

Node number 615: 
-----
TO

Node number 616: 
-----
non sondable

Node number 617: 
-----
TO

Node number 618: 
-----
non sondable

Node number 619: 
-----
TO

Node number 620: 
-----
non sondable

Node number 621: 
-----
TO

Node number 622: 
-----
non sondable

Node number 623: 
-----
TO

Node number 624: 
-----
non sondable

Node number 625: 
-----
TO

Node number 626: 
-----
non sondable

Node number 627: 
-----
TO

Node number 628: 
-----
non sondable

Node number 629: 
-----
TO

Node number 630: 
-----
non sondable

Node number 631: 
-----
TO

Node number 632: 
-----
non sondable

Node number 633: 
-----
TO

Node number 634: 
-----
non sondable

Node number 635: 
-----
TO

Node number 636: 
-----
non sondable

Node number 637: 
-----
TO

Node number 638: 
-----
non sondable

Node number 639: 
-----
TO

Node number 640: 
-----
non sondable

Node number 641: 
-----
TO

Node number 642: 
-----
non sondable

Node numbe


Node number 924: 
-----
non sondable

Node number 925: 
-----
TO

Node number 926: 
-----
non sondable

Node number 927: 
-----
TO

Node number 928: 
-----
non sondable

Node number 929: 
-----
TO

Node number 930: 
-----
non sondable

Node number 931: 
-----
TO

Node number 932: 
-----
non sondable

Node number 933: 
-----
TO

Node number 934: 
-----
non sondable

Node number 935: 
-----
TO

Node number 936: 
-----
non sondable

Node number 937: 
-----
TO

Node number 938: 
-----
non sondable

Node number 939: 
-----
TO

Node number 940: 
-----
non sondable

Node number 941: 
-----
TO

Node number 942: 
-----
non sondable

Node number 943: 
-----
TO

Node number 944: 
-----
non sondable

Node number 945: 
-----
TO

Node number 946: 
-----
non sondable

Node number 947: 
-----
TO

Node number 948: 
-----
non sondable

Node number 949: 
-----
TO

Node number 950: 
-----
non sondable

Node number 951: 
-----
TO

Node number 952: 
-----
non sondable

Node number 953: 
-----
TO

Node numb

Node number 1227: 
-----
TO

Node number 1228: 
-----
non sondable

Node number 1229: 
-----
TO

Node number 1230: 
-----
non sondable

Node number 1231: 
-----
TO

Node number 1232: 
-----
non sondable

Node number 1233: 
-----
TO

Node number 1234: 
-----
non sondable

Node number 1235: 
-----
TO

Node number 1236: 
-----
non sondable

Node number 1237: 
-----
TO

Node number 1238: 
-----
non sondable

Node number 1239: 
-----
TO

Node number 1240: 
-----
non sondable

Node number 1241: 
-----
TO

Node number 1242: 
-----
non sondable

Node number 1243: 
-----
TO

Node number 1244: 
-----
non sondable

Node number 1245: 
-----
TO

Node number 1246: 
-----
non sondable

Node number 1247: 
-----
TO

Node number 1248: 
-----
non sondable

Node number 1249: 
-----
TO

Node number 1250: 
-----
non sondable

Node number 1251: 
-----
TO

Node number 1252: 
-----
non sondable

Node number 1253: 
-----
TO

Node number 1254: 
-----
non sondable

Node number 1255: 
-----
TO

Node number 1256: 


...

Node number 11322: 
-----
non sondable

Node number 11323: 
-----
TO

Node number 11324: 
-----
non sondable

Node number 11325: 
-----
TO

Node number 11326: 
-----
non sondable

Node number 11327: 
-----
TO

Node number 11328: 
-----
non sondable

Node number 11329: 
-----
TO

Node number 11330: 
-----
non sondable

Node number 11331: 
-----
TO

Node number 11332: 
-----
non sondable

Node number 11333: 
-----
TO

Node number 11334: 
-----
non sondable

Node number 11335: 
-----
TO

Node number 11336: 
-----
non sondable

Node number 11337: 
-----
TO

Node number 11338: 
-----
non sondable

Node number 11339: 
-----
TO

Node number 11340: 
-----
non sondable

Node number 11341: 
-----
TO

Node number 11342: 
-----
non sondable

Node number 11343: 
-----
TO

Node number 11344: 
-----
non sondable

Node number 11345: 
-----
TO

Node number 11346: 
-----
non sondable

Node number 11347: 
-----
TO

Node number 11348: 
-----
non sondable

Node number 11349: 
-----
TO

Node number 11350: 


Node number 11615: 
-----
non sondable

Node number 11616: 
-----
TO

Node number 11617: 
-----
non sondable

Node number 11618: 
-----
TO

Node number 11619: 
-----
non sondable

Node number 11620: 
-----
TO

Node number 11621: 
-----
non sondable

Node number 11622: 
-----
TO

Node number 11623: 
-----
non sondable

Node number 11624: 
-----
TO

Node number 11625: 
-----
non sondable

Node number 11626: 
-----
TO

Node number 11627: 
-----
non sondable

Node number 11628: 
-----
TO

Node number 11629: 
-----
non sondable

Node number 11630: 
-----
TO

Node number 11631: 
-----
non sondable

Node number 11632: 
-----
TO

Node number 11633: 
-----
non sondable

Node number 11634: 
-----
TO

Node number 11635: 
-----
non sondable

Node number 11636: 
-----
TO

Node number 11637: 
-----
non sondable

Node number 11638: 
-----
TO

Node number 11639: 
-----
non sondable

Node number 11640: 
-----
TO

Node number 11641: 
-----
non sondable

Node number 11642: 
-----
TO

Node number 11643: 


Node number 12055: 
-----
non sondable

Node number 12056: 
-----
TO

Node number 12057: 
-----
non sondable

Node number 12058: 
-----
TO

Node number 12059: 
-----
non sondable

Node number 12060: 
-----
TO

Node number 12061: 
-----
non sondable

Node number 12062: 
-----
TO

Node number 12063: 
-----
non sondable

Node number 12064: 
-----
TO

Node number 12065: 
-----
non sondable

Node number 12066: 
-----
TO

Node number 12067: 
-----
non sondable

Node number 12068: 
-----
TO

Node number 12069: 
-----
non sondable

Node number 12070: 
-----
TO

Node number 12071: 
-----
non sondable

Node number 12072: 
-----
TO

Node number 12073: 
-----
non sondable

Node number 12074: 
-----
TO

Node number 12075: 
-----
non sondable

Node number 12076: 
-----
TO

Node number 12077: 
-----
non sondable

Node number 12078: 
-----
TO

Node number 12079: 
-----
non sondable

Node number 12080: 
-----
TO

Node number 12081: 
-----
non sondable

Node number 12082: 
-----
TO

Node number 12083: 


Node number 12348: 
-----
TO

Node number 12349: 
-----
non sondable

Node number 12350: 
-----
TO

Node number 12351: 
-----
non sondable

Node number 12352: 
-----
TO

Node number 12353: 
-----
non sondable

Node number 12354: 
-----
TO

Node number 12355: 
-----
non sondable

Node number 12356: 
-----
TO

Node number 12357: 
-----
non sondable

Node number 12358: 
-----
TO

Node number 12359: 
-----
non sondable

Node number 12360: 
-----
TO

Node number 12361: 
-----
non sondable

Node number 12362: 
-----
TO

Node number 12363: 
-----
non sondable

Node number 12364: 
-----
TO

Node number 12365: 
-----
non sondable

Node number 12366: 
-----
TO

Node number 12367: 
-----
non sondable

Node number 12368: 
-----
TO

Node number 12369: 
-----
non sondable

Node number 12370: 
-----
TO

Node number 12371: 
-----
non sondable

Node number 12372: 
-----
TO

Node number 12373: 
-----
non sondable

Node number 12374: 
-----
TO

Node number 12375: 
-----
non sondable

Node number 12376: 


Node number 12782: 
-----
non sondable

Node number 12783: 
-----
TO

Node number 12784: 
-----
non sondable

Node number 12785: 
-----
TO

Node number 12786: 
-----
non sondable

Node number 12787: 
-----
TO

Node number 12788: 
-----
non sondable

Node number 12789: 
-----
TO

Node number 12790: 
-----
non sondable

Node number 12791: 
-----
TO

Node number 12792: 
-----
non sondable

Node number 12793: 
-----
TO

Node number 12794: 
-----
non sondable

Node number 12795: 
-----
TO

Node number 12796: 
-----
non sondable

Node number 12797: 
-----
TO

Node number 12798: 
-----
non sondable

Node number 12799: 
-----
TO

Node number 12800: 
-----
non sondable

Node number 12801: 
-----
TO

Node number 12802: 
-----
non sondable

Node number 12803: 
-----
TO

Node number 12804: 
-----
non sondable

Node number 12805: 
-----
TO

Node number 12806: 
-----
TO

FINISHED

******

Optimal value = 12593.0

Optimal x=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1